<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multistock_td3_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y protobuf tensorflow gym keras torch torchvision torchaudio || true
!pip install protobuf==3.20.3 tensorflow==2.18.0 stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib matplotlib pandas numpy scipy scikit-learn imblearn
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [3]:
import os
import time
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# === Configuration ===
TICKERS = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]
INTERVAL = "1h"
PERIOD_DAYS = 720
SAVE_PATH = "/content/drive/MyDrive/trading_data/"
os.makedirs(SAVE_PATH, exist_ok=True)

# === Download Function ===
def download_data(ticker, retries=3, sleep_time=10):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=PERIOD_DAYS)
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    for attempt in range(1, retries + 1):
        try:
            print(f"⏳ Attempt {attempt}: {ticker} from {start_str} to {end_str}")
            df = yf.download(ticker, start=start_str, end=end_str, interval=INTERVAL, progress=False)
            if df.empty:
                raise ValueError("Downloaded DataFrame is empty.")
            df.reset_index(inplace=True)
            df['Symbol'] = ticker
            df['Datetime'] = pd.to_datetime(df['Datetime'] if 'Datetime' in df.columns else df['Date'])
            return df
        except Exception as e:
            print(f" Error for {ticker}: {e}. Retrying in {sleep_time} seconds...")
            time.sleep(sleep_time)
    return None

# === Feature Engineering Function (Fixed Stoch) ===
def compute_features(df):
    df = df.copy()

    # Drop duplicate columns if any
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    # Basic Indicators
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']

    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()

    #  FIXED Stoch: Make sure result is a Series, not DataFrame
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100

    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()

    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (typical_price - df['SMA_20']) / (0.015 * df['STD_20'])

    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()

    delta = df['Close'].diff()
    up = delta.clip(lower=0).rolling(14).mean()
    down = -delta.clip(upper=0).rolling(14).mean()
    df['RSI'] = 100 - (100 / (1 + up / down))

    true_range = pd.concat([
        df['High'] - df['Low'],
        (df['High'] - df['Close'].shift()).abs(),
        (df['Low'] - df['Close'].shift()).abs()
    ], axis=1).max(axis=1)
    df['ATR'] = true_range.rolling(14).mean()

    df['Volatility'] = df['Close'].pct_change().rolling(20).std()
    df['Return'] = (df['Close'].shift(-10) - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)

    df.dropna(inplace=True)
    return df

# === Run All Tickers ===
all_data = []
for ticker in TICKERS:
    print(f"\n Processing {ticker}")
    df_raw = download_data(ticker)
    if df_raw is not None and not df_raw.empty:
        try:
            df_feat = compute_features(df_raw)
            all_data.append(df_feat)
        except Exception as e:
            print(f" Feature engineering failed for {ticker}: {e}")
    else:
        print(f" Skipped {ticker}, no data downloaded.")

# === Save Combined Data ===
if all_data:
    df_all = pd.concat(all_data, ignore_index=True)
    df_all.to_csv(os.path.join(SAVE_PATH, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f"\n Saved dataset to {SAVE_PATH}multi_stock_feature_engineered_dataset.csv")
else:
    print(" No usable data for any ticker.")



 Processing AAPL
⏳ Attempt 1: AAPL from 2023-06-22 to 2025-06-11
YF.download() has changed argument auto_adjust default to True

 Processing TSLA
⏳ Attempt 1: TSLA from 2023-06-22 to 2025-06-11

 Processing MSFT
⏳ Attempt 1: MSFT from 2023-06-22 to 2025-06-11

 Processing GOOGL
⏳ Attempt 1: GOOGL from 2023-06-22 to 2025-06-11

 Processing AMZN
⏳ Attempt 1: AMZN from 2023-06-22 to 2025-06-11

 Processing NVDA
⏳ Attempt 1: NVDA from 2023-06-22 to 2025-06-11

 Processing META
⏳ Attempt 1: META from 2023-06-22 to 2025-06-11

 Processing BRK-B
⏳ Attempt 1: BRK-B from 2023-06-22 to 2025-06-11

 Processing JPM
⏳ Attempt 1: JPM from 2023-06-22 to 2025-06-11

 Processing JNJ
⏳ Attempt 1: JNJ from 2023-06-22 to 2025-06-11

 Processing XOM
⏳ Attempt 1: XOM from 2023-06-22 to 2025-06-11

 Processing V
⏳ Attempt 1: V from 2023-06-22 to 2025-06-11

 Processing PG
⏳ Attempt 1: PG from 2023-06-22 to 2025-06-11

 Processing UNH
⏳ Attempt 1: UNH from 2023-06-22 to 2025-06-11

 Processing MA
⏳ Attempt 1

In [8]:
# === Mount Google Drive ===
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
!rm -rf /content/drive

In [9]:
# === Imports ===
import gc
import json
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from datetime import datetime
from stable_baselines3 import TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.spaces import Box
from gym_anytrading.envs import StocksEnv

SAVE_DIR = "/content/drive/MyDrive/Results_May_2025/results_td3_walkforward/td3_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/results_td3_walkforward"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# === Config ===
test_mode = False
test_ticker = 'AAPL'
gpu_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\u2705 Using device: {gpu_device}")

train_start = pd.to_datetime("2023-06-01").tz_localize("UTC")
train_end = pd.to_datetime("2024-06-01").tz_localize("UTC")
test_start = pd.to_datetime("2024-06-01").tz_localize("UTC")
test_end = pd.to_datetime("2025-04-28").tz_localize("UTC")

TICKERS = [test_ticker] if test_mode else [
    'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

# === Load Dataset ===
df_all = pd.read_csv("/content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv")
df_all['Datetime'] = pd.to_datetime(df_all['Datetime'])
df_all['Datetime'] = df_all['Datetime'].dt.tz_convert("UTC")

# === Environment ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df, frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def step(self, action):
        action = action[0] if isinstance(action, np.ndarray) else action
        if action < -0.3:
            discrete_action = 0
        elif action > 0.3:
            discrete_action = 1
        else:
            discrete_action = 2
        return super().step(discrete_action)

# === Training Loop ===
results = []

for symbol in TICKERS:
    model_path = os.path.join(SAVE_DIR, f"{symbol}_td3_model.zip")
    features_path = os.path.join(SAVE_DIR, f"{symbol}_features.json")

    if os.path.exists(model_path):
        print(f"\u2705 Skipping {symbol}, model already exists.")
        continue

    print(f"\n\U0001F4C8 Processing {symbol}")
    df = df_all[df_all['Symbol'] == symbol].copy()
    if df.empty:
        print(f"\u274C No data for {symbol}")
        continue

    df = df.sort_values("Datetime")
    df_train = df[(df['Datetime'] >= train_start) & (df['Datetime'] < train_end)].copy()
    df_test = df[(df['Datetime'] >= test_start) & (df['Datetime'] <= test_end)].copy()

    if len(df_train) < 300 or len(df_test) < 100:
        print(f" Skipping {symbol}: insufficient data.")
        continue

    feature_cols = [col for col in df.columns if col not in ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Symbol', 'Target', 'Return']]
    with open(features_path, 'w') as f:
        json.dump(feature_cols, f)

    env_train = DummyVecEnv([lambda: ContinuousTradingEnv(df_train, (50, len(df_train)), 10)])
    env_test = DummyVecEnv([lambda: ContinuousTradingEnv(df_test, (50, len(df_test)), 10)])

    model = TD3("MlpPolicy", env_train, learning_rate=1e-4, buffer_size=100000, batch_size=256,
                gamma=0.995, policy_delay=2, verbose=0, device=gpu_device)

    model.learn(total_timesteps=100_000)
    model.save(model_path)
    print(f"\u2705 Model saved: {model_path}")

    obs = env_test.reset()
    portfolio, trade_log, buy_price = [], [], None
    balance, position = 100000, 0
    close_prices = df_test['Close'].values

    for i in range(len(df_test) - 50):
        action, _ = model.predict(obs)
        obs, _, done, _ = env_test.step(action)[:4]
        a = float(np.squeeze(action))
        price = close_prices[i + 50]

        if a > 0.3 and buy_price is None:
            buy_price = price
            position = balance / price
            balance = 0
            trade_log.append("BUY")
        elif a < -0.3 and buy_price is not None:
            balance = position * price
            position = 0
            buy_price = None
            trade_log.append("SELL")
        else:
            trade_log.append("HOLD")

        portfolio.append(balance if balance > 0 else position * price)
        if done[0]:
            break

    final_value = portfolio[-1]
    cumulative_return = ((final_value / 100000) - 1) * 100
    daily_returns = pd.Series(portfolio).pct_change().fillna(0)
    sharpe = (daily_returns.mean() / (daily_returns.std() + 1e-6)) * np.sqrt(252)
    drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100

    # Save plot
    plt.figure(figsize=(10, 4))
    plt.plot(portfolio, label="TD3 Portfolio")
    plt.title(f"{symbol} - TD3 Performance")
    plt.xlabel("Timestep")
    plt.ylabel("Portfolio Value")
    plt.grid(True)
    plt.legend()
    plt.savefig(os.path.join(RESULTS_DIR, f"{symbol}_td3_plot.png"))
    plt.close()

    results.append({
        "Ticker": symbol,
        "Sharpe": round(sharpe, 4),
        "Drawdown": round(drawdown, 2),
        "Return": round(cumulative_return, 2),
        "Final_Portfolio": round(final_value, 2)
    })

    del model, env_train, env_test
    gc.collect()

# === Save Results ===
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(RESULTS_DIR, "td3_datebased_walkforward_summary.csv"), index=False)
print("\U0001F4BE Results saved to Google Drive")


 Using device: cuda

 Processing CVX
 Model saved: /content/drive/MyDrive/Results_May_2025/results_td3_walkforward/td3_walkforward_models/CVX_td3_model.zip

 Processing IBM
 Model saved: /content/drive/MyDrive/Results_May_2025/results_td3_walkforward/td3_walkforward_models/IBM_td3_model.zip

 Processing GE
 Model saved: /content/drive/MyDrive/Results_May_2025/results_td3_walkforward/td3_walkforward_models/GE_td3_model.zip

 Processing SBUX
 Model saved: /content/drive/MyDrive/Results_May_2025/results_td3_walkforward/td3_walkforward_models/SBUX_td3_model.zip

 Processing ORCL
 Model saved: /content/drive/MyDrive/Results_May_2025/results_td3_walkforward/td3_walkforward_models/ORCL_td3_model.zip
💾 Results saved to Google Drive
